# 0. Imports and Setting up Anthropic API Client

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-dotenv

import os
import dotenv

dotenv.load_dotenv('/content/drive/MyDrive/.env')

True

In [3]:
# Load Prompts and Problem Description
# Variables Prompt
prompt11_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt11_MathematicalModel.txt'

# Objective Prompt
prompt12_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt12_MathematicalModel.txt'

# Constraint Prompt
prompt13_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt13_MathematicalModel.txt'

# Code Prompt
prompt2_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt2_PyomoCode.txt'
problem_desc_path = '/content/drive/MyDrive/Thesis/ProblemDescriptions/IP/IP4.txt'

prompt11_file = open(prompt11_path, "r")
prompt12_file = open(prompt12_path, "r")
prompt13_file = open(prompt13_path, "r")
prompt2_file = open(prompt2_path, "r")
problem_desc_file = open(problem_desc_path, "r")

prompt11 = prompt11_file.read()
print("Prompt 1.1 (Variables):\n", prompt11)

prompt12 = prompt12_file.read()
print("Prompt 1.2 (Objctive):\n", prompt12)

prompt13 = prompt13_file.read()
print("Prompt 1.3 (Constraints):\n", prompt13)

prompt2 = prompt2_file.read()
print("Prompt 2:\n", prompt2)

problem_desc = problem_desc_file.read()
print("Problem Description:\n", problem_desc)

Prompt 1.1 (Variables):
 Please formulate only the variables for this mathematical optimization problem. 
Prompt 1.2 (Objctive):
 Please formulate only the objective function for this mathematical optimization problem. 
Prompt 1.3 (Constraints):
 Please formulate only the constraints for this mathematical optimization problem. 
Prompt 2:
 Please write a python pyomo code for this optimization problem.
Use sample data where needed.
Indicate where you use sample data.
Problem Description:
 Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three kinds of goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals: 
You have a target of selling 5500 units. 
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target. 
On average, your full-time employees will sell 5 records per hour and your part-time employees will sell 2 records per hour.

Employe

In [4]:
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.7/862.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [5]:
# Importing Anthropic & Setting Headers
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

# 1. Prompt 1.1: Create Variables for Mathematical Model

In [6]:
message11 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt11,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": problem_desc}
    ]
)

response_p11 = message11.content[0].text

In [7]:
# Print response
print(response_p11)

Here are the variables for the mathematical optimization problem:

Decision Variables:
- F: Number of hours worked by full-time employees
- P: Number of hours worked by part-time employees

Deviation Variables:
- d1+: Positive deviation from sales target (units)
- d1-: Negative deviation from sales target (units)
- d2+: Positive deviation from full-time employee hours target
- d2-: Negative deviation from full-time employee hours target
- d3+: Positive deviation from part-time employee hours target
- d3-: Negative deviation from part-time employee hours target
- d4+: Positive deviation from full-time employee overtime hours target
- d4-: Negative deviation from full-time employee overtime hours target
- d5+: Positive deviation from total overtime hours (for both full-time and part-time employees)

Priority Factors:
- P1: Priority for deviations from sales target
- P2: Priority for deviations from full-time employee overtime hours target
- P3: Priority for deviations from full-time empl

# 1. Prompt 1.2: Create Objective for Mathematical Model

In [8]:
message12 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt12,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": (problem_desc + response_p11)}
    ]
)

response_p12 = message12.content[0].text

In [9]:
# Print response
print(response_p12)

Objective Function:

Minimize: 
P1 * (d1+ + d1-) + P2 * (d4+ + d4-) + P3 * 2 * (d2+ + d2-) + P4 * (d3+ + d3-) + P5 * (d5+ + 3 * (P - 320))

Explanation:
- P1 * (d1+ + d1-): Minimizes the deviations from the sales target, with priority P1.
- P2 * (d4+ + d4-): Minimizes the deviations from the full-time employee overtime hours target, with priority P2.
- P3 * 2 * (d2+ + d2-): Minimizes the deviations from the full-time employee hours target, with priority P3. The factor 2 represents the relative importance of utilizing full-time employee hours compared to part-time employee hours.
- P4 * (d3+ + d3-): Minimizes the deviations from the part-time employee hours target, with priority P4.
- P5 * (d5+ + 3 * (P - 320)): Minimizes the total overtime hours for both full-time and part-time employees, with priority P5. The term 3 * (P - 320) represents the overtime hours for part-time employees, considering the relative cost of part-time employee overtime hours compared to full-time employee overti

# 1. Prompt 1.3: Create Constraints for Mathematical Model


In [10]:
message13 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt13,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": (problem_desc + response_p11 + response_p12)}
    ]
)

response_p13 = message13.content[0].text

In [11]:
# Print response
print(response_p13)

Here are the constraints for the mathematical optimization problem:

Sales Constraint:
5 * F + 2 * P + d1- - d1+ = 5500

Explanation: 
This constraint ensures that the total sales achieved by full-time and part-time employees, plus any deviations, meet the sales target of 5500 units. The sales per hour for full-time employees (5) and part-time employees (2) are multiplied by their respective hours worked (F and P).

Full-time Employee Hours Constraint:
F + d2- - d2+ = 800

Explanation:
This constraint ensures that the total hours worked by full-time employees, plus any deviations, meet the full-time employee hours target of 800 hours.

Part-time Employee Hours Constraint:
P + d3- - d3+ = 320

Explanation:
This constraint ensures that the total hours worked by part-time employees, plus any deviations, meet the part-time employee hours target of 320 hours.

Full-time Employee Overtime Hours Constraint:
F - 800 + d4- - d4+ = 100

Explanation:
This constraint ensures that the overtime hour

# 2. Prompt 2: Write Code based on Mathematical Model

In case more than the component of each step was generated, use "last instance"/ "newest version" of the components.

Example:
Prompt 1.1 (Variables): Model generates only variables
Prompt 1.2 (Objectvie): Model generates objective and constraints
Prompt 1.3 (Constraints): Model generates only constraints

Use Variables from step 1.1, use objective from 1.2 and use constraints from 1.3 (since constraints version 1.3 is more recent than constraints version 1.2).

This set up allows for potential self-correction of the model throughout the three prompts.

Correctness of each component is still generated in the respective step, so self-correction is independent of this.

In [12]:
message2 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt2,
    max_tokens=4069,
    messages=[
        {"role": "user", "content": (response_p11 + response_p12 + response_p13)}
    ]
)

response_p2 = message2.content[0].text

In [13]:
# Print response
print(response_p2)

Here's a Python Pyomo code for the optimization problem:

```python
from pyomo.environ import *

# Create the model
model = ConcreteModel()

# Define the decision variables
model.F = Var(within=NonNegativeReals)  # Number of hours worked by full-time employees
model.P = Var(within=NonNegativeReals)  # Number of hours worked by part-time employees

# Define the deviation variables
model.d1_plus = Var(within=NonNegativeReals)  # Positive deviation from sales target
model.d1_minus = Var(within=NonNegativeReals)  # Negative deviation from sales target
model.d2_plus = Var(within=NonNegativeReals)  # Positive deviation from full-time employee hours target
model.d2_minus = Var(within=NonNegativeReals)  # Negative deviation from full-time employee hours target
model.d3_plus = Var(within=NonNegativeReals)  # Positive deviation from part-time employee hours target
model.d3_minus = Var(within=NonNegativeReals)  # Negative deviation from part-time employee hours target
model.d4_plus = Var(within=N

# 4. Input Problem Data and Test Model Code

In [14]:
# Installing pyomo nd solver
!pip install -q pyomo
!pip install pandas
!apt-get install -y -qq glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.0 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [16]:
from pyomo.environ import *

# Create the model
model = ConcreteModel()

# Define the decision variables
model.F = Var(within=NonNegativeReals)  # Number of hours worked by full-time employees
model.P = Var(within=NonNegativeReals)  # Number of hours worked by part-time employees

# Define the deviation variables
model.d1_plus = Var(within=NonNegativeReals)  # Positive deviation from sales target
model.d1_minus = Var(within=NonNegativeReals)  # Negative deviation from sales target
model.d2_plus = Var(within=NonNegativeReals)  # Positive deviation from full-time employee hours target
model.d2_minus = Var(within=NonNegativeReals)  # Negative deviation from full-time employee hours target
model.d3_plus = Var(within=NonNegativeReals)  # Positive deviation from part-time employee hours target
model.d3_minus = Var(within=NonNegativeReals)  # Negative deviation from part-time employee hours target
model.d4_plus = Var(within=NonNegativeReals)  # Positive deviation from full-time employee overtime hours target
model.d4_minus = Var(within=NonNegativeReals)  # Negative deviation from full-time employee overtime hours target
model.d5_plus = Var(within=NonNegativeReals)  # Positive deviation from total overtime hours

# Define the priority factors (sample data)
P1 = 5  # Priority for deviations from sales target
P2 = 4   # Priority for deviations from full-time employee overtime hours target
P3 = 3   # Priority for deviations from full-time employee hours target
P4 = 2   # Priority for deviations from part-time employee hours target
P5 = 1   # Priority for deviations from total overtime hours

# Define the objective function
model.objective = Objective(expr=P1 * (model.d1_plus + model.d1_minus) +
                                 P2 * (model.d4_plus + model.d4_minus) +
                                 P3 * 2 * (model.d2_plus + model.d2_minus) +
                                 P4 * (model.d3_plus + model.d3_minus) +
                                 P5 * (model.d5_plus + 3 * (model.P - 320)),
                            sense=minimize)

# Define the constraints
model.sales_constraint = Constraint(expr=5 * model.F + 2 * model.P + model.d1_minus - model.d1_plus == 5500)
model.fulltime_hours_constraint = Constraint(expr=model.F + model.d2_minus - model.d2_plus == 800)
model.parttime_hours_constraint = Constraint(expr=model.P + model.d3_minus - model.d3_plus == 320)
model.fulltime_overtime_constraint = Constraint(expr=model.F - 800 + model.d4_minus - model.d4_plus == 100)
model.total_overtime_constraint = Constraint(expr=(model.F - 800) + (model.P - 320) + model.d5_minus - model.d5_plus == 0)

# Solve the model
results = SolverFactory('glpk').solve(model)

# Print the results
print("Objective value:", model.objective())
print("Full-time employee hours (F):", model.F())
print("Part-time employee hours (P):", model.P())

AttributeError: 'ConcreteModel' object has no attribute 'd5_minus'

In [ ]:
print("d1^-:", model.d1_minus())
print("d1^+:", model.d1_plus())
print("d2^-:", model.d2_minus())
print("d2^+:", model.d2_plus())
print("d3^-:", model.d3_minus())
print("d3^+:", model.d3_plus())
print("d4^-:", model.d4_minus())
print("d4^+:", model.d4_plus())
print("d5^+:", model.d5_plus())

d1^-: 0.0
d1^+: 0.0
d2^-: 0.0
d2^+: 172.0
d3^-: 0.0
d3^+: 0.0
d4^-: 0.0
d4^+: 72.0
d5^+: 172.0


# 5. Correct The Model Code to Test Mathematical Model (if applicable)

In [18]:
from pyomo.environ import *

# Create the model
model = ConcreteModel()

# Define the decision variables
model.F = Var(within=NonNegativeReals)  # Number of hours worked by full-time employees
model.P = Var(within=NonNegativeReals)  # Number of hours worked by part-time employees

# Define the deviation variables
model.d1_plus = Var(within=NonNegativeReals)  # Positive deviation from sales target
model.d1_minus = Var(within=NonNegativeReals)  # Negative deviation from sales target
model.d2_plus = Var(within=NonNegativeReals)  # Positive deviation from full-time employee hours target
model.d2_minus = Var(within=NonNegativeReals)  # Negative deviation from full-time employee hours target
model.d3_plus = Var(within=NonNegativeReals)  # Positive deviation from part-time employee hours target
model.d3_minus = Var(within=NonNegativeReals)  # Negative deviation from part-time employee hours target
model.d4_plus = Var(within=NonNegativeReals)  # Positive deviation from full-time employee overtime hours target
model.d4_minus = Var(within=NonNegativeReals)  # Negative deviation from full-time employee overtime hours target
model.d5_plus = Var(within=NonNegativeReals)  # Positive deviation from total overtime hours
model.d5_minus = Var(within=NonNegativeReals)  # Positive deviation from total overtime hours

# Define the priority factors (sample data)
P1 = 5  # Priority for deviations from sales target
P2 = 4   # Priority for deviations from full-time employee overtime hours target
P3 = 3   # Priority for deviations from full-time employee hours target
P4 = 2   # Priority for deviations from part-time employee hours target
P5 = 1   # Priority for deviations from total overtime hours

# Define the objective function
model.objective = Objective(expr=P1 * (model.d1_plus + model.d1_minus) +
                                 P2 * (model.d4_plus + model.d4_minus) +
                                 P3 * 2 * (model.d2_plus + model.d2_minus) +
                                 P4 * (model.d3_plus + model.d3_minus) +
                                 P5 * (model.d5_plus + 3 * (model.P - 320)),
                            sense=minimize)

# Define the constraints
model.sales_constraint = Constraint(expr=5 * model.F + 2 * model.P + model.d1_minus - model.d1_plus == 5500)
model.fulltime_hours_constraint = Constraint(expr=model.F + model.d2_minus - model.d2_plus == 800)
model.parttime_hours_constraint = Constraint(expr=model.P + model.d3_minus - model.d3_plus == 320)
model.fulltime_overtime_constraint = Constraint(expr=model.F - 800 + model.d4_minus - model.d4_plus == 100)
model.total_overtime_constraint = Constraint(expr=(model.F - 800) + (model.P - 320) + model.d5_minus - model.d5_plus == 0)

# Solve the model
results = SolverFactory('glpk').solve(model)

# Print the results
print("Objective value:", model.objective())
print("Full-time employee hours (F):", model.F())
print("Part-time employee hours (P):", model.P())
print("d1^-:", model.d1_minus())
print("d1^+:", model.d1_plus())
print("d2^-:", model.d2_minus())
print("d2^+:", model.d2_plus())
print("d3^-:", model.d3_minus())
print("d3^+:", model.d3_plus())
print("d4^-:", model.d4_minus())
print("d4^+:", model.d4_plus())
print("d5^+:", model.d5_plus())

Objective value: 1492.0
Full-time employee hours (F): 972.0
Part-time employee hours (P): 320.0
d1^-: 0.0
d1^+: 0.0
d2^-: 0.0
d2^+: 172.0
d3^-: 0.0
d3^+: 0.0
d4^-: 0.0
d4^+: 72.0
d5^+: 172.0
